In [1]:
import sys
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
import datetime
from psaw import PushshiftAPI
from __future__ import annotations
import os
import json
from tqdm import tqdm

sys.path.append("..")
from src.read_books import mentioned_chars

In [5]:
api = PushshiftAPI()
features = ["title", "id", "score", "author", "created_utc", "selftext"]
subreddit = "asoiaf"
date1 = int(datetime.datetime(2015,7,16).timestamp())
date2 = int(datetime.datetime(2019,4,14).timestamp())
gen = api.search_submissions(subreddit = subreddit, after = date1, before = date2, q='', filter = features, limit = 3 * 10**5)

In [6]:
results = list(gen)

C:\Users\Oskar Wiese\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Oskar Wiese\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [7]:

ids, title, score, author, created_utc, selftext = [], [], [], [], [], []
for i in range(len(results)):
    try:
        selftext.append(results[i].d_['selftext'])
        ids.append(i)
        title.append(results[i].d_['title'])
        score.append(results[i].d_['score'])
        created_utc.append(results[i].d_['created_utc'])
        author.append(results[i].d_['author'])
    except KeyError:
        pass
        

data = pd.DataFrame(
{
    "id" : ids,
    "title" : title,
    "score" : score,
    "created" : created_utc,
    "author" : author,
    "selftext" : selftext
}, index = None)

In [28]:
# To see amount of posts that acutally has text in selftext
def get_usefull_data(data):
    for i in tqdm(range(len(data))):
        if data['selftext'][i] == '':
            data.drop(data.index[i])
        if data['selftext'][i] == '[removed]':
            data.drop(data.index[i])
    return data.dropna()

def get_postw_names():
    text_list = [text for text in data['names'] if text != [] ]
    return len(text_list)

data1 = get_usefull_data(data)
data

100%|█████████████████████████████████████████████████████████████████████████| 117742/117742 [13:45<00:00, 142.64it/s]


,id,title,score,created,author,selftext,names
0,0,How Game of Thrones Lost Interest in Tyrion La...,1,1555192642,Euphoric_Translator,,[]
1,1,(Spoilers Extended) The meeting that I'm waiti...,376,1555191033,JRR_STARK,,[]
2,2,Unborn Lannister,1,1555191006,kcconk,[removed],[]
3,3,(Spoilers extended) This sub is going to explo...,7,1555189915,seperationsunday,2019. r/asoiaf is about to E.X.P.L.O.D.E\n\nAr...,[]
4,4,(Spoilers extended) For all those that love to...,1,1555188219,KingPoTheThird,,[]
...,...,...,...,...,...,...,...
117737,118855,(Spoilers All) Need help finding Stannis quote...,1,1436999414,user1444,Can you guys list every scene you can think of...,"[Stannis Baratheon, Stannis Baratheon]"
117738,118856,(Spoilers All) I didn't realize how much fores...,62,1436999159,Compeau,This is from a Bran chapter in AGOT.\n\n&gt;He...,"[Brandon 'Bran' Stark, Sansa Stark, Arya Stark..."
117739,118857,Doesn't Tyrion have any bastards?,1,1436998933,[deleted],,[]
117740,118858,(Spoilers All) AGOT Sansa chapter with the for...,0,1436998367,itsmearyastark,"I don't know if this has been mentioned here, ...","[Sansa Stark, Sansa Stark]"


In [37]:
data1 = data[data['selftext'] != '']
data1 = data1[data1['selftext'] != '[removed]']
data1 = data1.dropna()
data1 = data1.drop('id', axis=1)
data1.to_csv('asoiaf_data_cleaned.csv',index=True)

In [17]:
test = mentioned_chars(data)
test

,id,title,score,created,author,selftext,names
0,0,How Game of Thrones Lost Interest in Tyrion La...,1,1555192642,Euphoric_Translator,,[]
1,1,(Spoilers Extended) The meeting that I'm waiti...,376,1555191033,JRR_STARK,,[]
2,2,Unborn Lannister,1,1555191006,kcconk,[removed],[]
3,3,(Spoilers extended) This sub is going to explo...,7,1555189915,seperationsunday,2019. r/asoiaf is about to E.X.P.L.O.D.E\n\nAr...,[]
4,4,(Spoilers extended) For all those that love to...,1,1555188219,KingPoTheThird,,[]
...,...,...,...,...,...,...,...
117737,118855,(Spoilers All) Need help finding Stannis quote...,1,1436999414,user1444,Can you guys list every scene you can think of...,"[Stannis Baratheon, Stannis Baratheon]"
117738,118856,(Spoilers All) I didn't realize how much fores...,62,1436999159,Compeau,This is from a Bran chapter in AGOT.\n\n&gt;He...,"[Brandon 'Bran' Stark, Sansa Stark, Arya Stark..."
117739,118857,Doesn't Tyrion have any bastards?,1,1436998933,[deleted],,[]
117740,118858,(Spoilers All) AGOT Sansa chapter with the for...,0,1436998367,itsmearyastark,"I don't know if this has been mentioned here, ...","[Sansa Stark, Sansa Stark]"


In [25]:
data1 = get_postw_names()


37724

In [21]:
data.to_csv('asoiaf_data.csv',index=True)

In [22]:
data

,id,title,score,created,author,selftext,names
0,0,How Game of Thrones Lost Interest in Tyrion La...,1,1555192642,Euphoric_Translator,,[]
1,1,(Spoilers Extended) The meeting that I'm waiti...,376,1555191033,JRR_STARK,,[]
2,2,Unborn Lannister,1,1555191006,kcconk,[removed],[]
3,3,(Spoilers extended) This sub is going to explo...,7,1555189915,seperationsunday,2019. r/asoiaf is about to E.X.P.L.O.D.E\n\nAr...,[]
4,4,(Spoilers extended) For all those that love to...,1,1555188219,KingPoTheThird,,[]
...,...,...,...,...,...,...,...
117737,118855,(Spoilers All) Need help finding Stannis quote...,1,1436999414,user1444,Can you guys list every scene you can think of...,"[Stannis Baratheon, Stannis Baratheon]"
117738,118856,(Spoilers All) I didn't realize how much fores...,62,1436999159,Compeau,This is from a Bran chapter in AGOT.\n\n&gt;He...,"[Brandon 'Bran' Stark, Sansa Stark, Arya Stark..."
117739,118857,Doesn't Tyrion have any bastards?,1,1436998933,[deleted],,[]
117740,118858,(Spoilers All) AGOT Sansa chapter with the for...,0,1436998367,itsmearyastark,"I don't know if this has been mentioned here, ...","[Sansa Stark, Sansa Stark]"


In [18]:
""" For fixing the data """
#%cd ..
# Requires that you cd up one level
!pwd
df_reddit = pd.read_csv(os.path.join("local_data/asoiaf_data_cleaned.csv"), header=0, index_col=0)
mentioned_chars(df_reddit, text_col="selftext")

/home/sorenwh/Nextcloud/semester6/soc/adpocalypse


,title,score,created,author,selftext,names
3,(Spoilers extended) This sub is going to explo...,7,1555189915,seperationsunday,2019. r/asoiaf is about to E.X.P.L.O.D.E\r\n\r...,[]
7,[SPOILERS EXTENDED] Miiri’s Prophecy,2,1555187335,ChayaFeige,Miiri Maaz Dur Prophecy decoded using Gilly’s ...,[]
8,(Spoilers Main) GOT Theory: You Can't Trust th...,47,1555186553,John-on-gliding,The events that led to the creation of the Whi...,[Brandon 'Bran' Stark]
10,(Spoilers extended)The Night King needs Bran,3,1555184407,dikziw,Ever since the episode where Bran is “marked” ...,"[Jon Snow, Brandon 'Bran' Stark]"
16,(Spoilers Extended) Ending theory: a Tudor Wes...,101,1555182090,5555512369874,ASOIAF has a number of historical inspirations...,"[Petyr 'Littlefinger' Baelish, Jon Snow, Varys]"
...,...,...,...,...,...,...
117736,(Spoilers All) Tv Show direction,17,1437000037,ShutYoFaceGrandma,"So, I was very unhappy about the Sansa switche...","[Sansa Stark, Stannis Baratheon, Ramsay Bolton]"
117737,(Spoilers All) Need help finding Stannis quote...,1,1436999414,user1444,Can you guys list every scene you can think of...,[Stannis Baratheon]
117738,(Spoilers All) I didn't realize how much fores...,62,1436999159,Compeau,This is from a Bran chapter in AGOT.\r\n\r\n&g...,"[Arya Stark, Sansa Stark, Sandor Clegane, Jon ..."
117740,(Spoilers All) AGOT Sansa chapter with the for...,0,1436998367,itsmearyastark,"I don't know if this has been mentioned here, ...","[Sansa Stark, Jon Snow]"


In [19]:
df_reddit = df_reddit.dropna()
df_reddit.to_csv('local_data/asoiaf_data_cleaned.csv',index=True)

,title,score,created,author,selftext,names
117736,(Spoilers All) Tv Show direction,17,1437000037,ShutYoFaceGrandma,"So, I was very unhappy about the Sansa switche...","[Sansa Stark, Stannis Baratheon, Ramsay Bolton]"
117737,(Spoilers All) Need help finding Stannis quote...,1,1436999414,user1444,Can you guys list every scene you can think of...,[Stannis Baratheon]
117738,(Spoilers All) I didn't realize how much fores...,62,1436999159,Compeau,This is from a Bran chapter in AGOT.\r\n\r\n&g...,"[Arya Stark, Sansa Stark, Sandor Clegane, Jon ..."
117740,(Spoilers All) AGOT Sansa chapter with the for...,0,1436998367,itsmearyastark,"I don't know if this has been mentioned here, ...","[Sansa Stark, Jon Snow]"
117741,(Spoilers All) What would you sacrifice to rea...,39,1436997975,[deleted],The Shadowbinders and Maegi claim that only bl...,[Melisandre]
